In [10]:
''' Handle Colab Stuff '''
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(r'./')
from argparse import Namespace
import torch
if not torch.device('cuda:0'):  raise IOError("Cannot find GPU")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!pip3 install -q --upgrade tensorboardX
%load_ext tensorboard
from tensorboardX import SummaryWriter

In [6]:
config = {
  "cuda": True, "do_train": True, "do_valid": False, "do_test": False,
  "data_path": "data/FB15k-betae",
  "negative_sample_size": 128,	# negative entities sampled per query
  "hidden_dim": 400,	# embedding dimension
  "gamma": 60.0,	# margin in the loss
  "batch_size": 512,	# batch size of queries
  "test_batch_size": 1,	# valid/test batch size
  "learning_rate": 0.0001,
  "cpu_num": 1,	# used to speed up torch.dataloader
  "save_path": "./saves",	# no need to set manually, will configure automatically
  "max_steps": 450001,  # maximum iterations to train
  "warm_up_steps": None,	# no need to set manually, will configure automatically

  "save_checkpoint_steps": 50000,  # save checkpoints every xx steps
  "valid_steps": 15000,	# evaluate validation queries every xx steps
  "log_steps": 100,	# train log every xx steps
  "test_log_steps": 1000,	# valid/test log every xx steps

  "nentity": 0,  # DO NOT MANUALLY SET
  "nrelation": 0,	# DO NOT MANUALLY SET

  "geo": "beta",	# the reasoning model, vec for GQE, box for Query2box, beta for BetaE
  "print_on_screen": False,	# print_on_screen

  "tasks": "1p.2p.3p.2i.3i.ip.pi.2in.3in.inp.pin.pni.2u.up",	# tasks connected by dot, refer to the BetaE paper for detailed meaning and structure of each task
  "seed": 0,	# random seed
  "beta_mode": "(1600,2)",	# (hidden_dim,num_layer for BetaE relational projection)
  "box_mode": "(none,0.02)",	# (offset activation,center_reg for Query2box, center_reg balances the in_box dist and out_box dist)
  "prefix": "logs",	# prefix of the log path
  "checkpoint_path": None,	# path for loading the checkpoints
  "evaluate_union": "DNF" # the way to evaluate union queries, transform it to disjunctive normal form (DNF or use the De Morgan\s laws (DM))
}
args = Namespace(**config)

In [7]:
''' Import library '''
from tensorboardX import SummaryWriter
from collections import defaultdict # unlike dict, return default if key not exist
from torch.utils.data import DataLoader
import numpy as np

''' Import helper function and classes'''
from betae.util import flatten_query, list2tuple, parse_time, set_global_seed, eval_tuple, set_logger, save_model, evaluate, log_metrics
from betae.dataloader import TestDataset, TrainDataset, SingledirectionalOneShotIterator
from betae.model import KGReasoning
''' Import query tasks '''
from betae.query import query_name_dict, name_query_dict, all_tasks

In [12]:
def main(args):
    set_global_seed(args.seed)
    tasks = args.tasks.split('.')
    cur_time = parse_time()

    ''' Generate Save path '''
    args.save_path = os.path.join(args.prefix, args.data_path.split('/')[-1], args.tasks.replace('.', '_'), args.geo)
    tmp_str = "g_{}_mode_{}".format(args.gamma, args.beta_mode.replace(',', '_'))
    args.save_path = args.checkpoint_path if args.checkpoint_path != None else os.path.join(args.save_path, tmp_str, cur_time)
    if not os.path.exists(args.save_path):
        os.makedirs(os.path.join('./', args.save_path))
    print ("logging to", args.save_path)

    ''' Generate Tensorboard files '''
    if not args.do_train: # if not training, then create tensorboard files in some tmp location
        writer = SummaryWriter('./logs-debug/unused-tb')
    else:
        writer = SummaryWriter(args.save_path)
    
    ''' Generate logging handle '''
    logger = set_logger(args)

    ''' Read in dataset info '''
    with open('%s/stats.txt'%args.data_path) as f:
        entrel = f.readlines()
        nentity = int(entrel[0].split(' ')[-1])
        nrelation = int(entrel[1].split(' ')[-1])
    args.nentity = nentity
    args.nrelation = nrelation
    
    ''' Loading Data from file '''
    logger.info('-------------------------------'*3)
    logger.info('Geo: %s' % args.geo)
    logger.info('Data Path: %s' % args.data_path)
    logger.info('#entity: %d' % nentity)
    logger.info('#relation: %d' % nrelation)
    logger.info('#max steps: %d' % args.max_steps)
    logger.info('Evaluate unoins using: %s' % args.evaluate_union)
    train_queries, train_answers, valid_queries, valid_hard_answers, valid_easy_answers, test_queries, test_hard_answers, test_easy_answers = load_data(args, tasks)        

    ''' Save training related info '''
    logger.info("Training info:")
    if args.do_train:
        for query_structure in train_queries:
            logger.info(query_name_dict[query_structure]+": "+str(len(train_queries[query_structure])))

        ''' Split query into path queries and other queries iterator '''
        train_path_queries = defaultdict(set)
        train_other_queries = defaultdict(set)
        path_list = ['1p', '2p', '3p']
        for query_structure in train_queries:
            if query_name_dict[query_structure] in path_list:
                train_path_queries[query_structure] = train_queries[query_structure]
            else:
                train_other_queries[query_structure] = train_queries[query_structure]
        train_path_queries = flatten_query(train_path_queries)
        train_path_iterator = SingledirectionalOneShotIterator(DataLoader(
                                    TrainDataset(train_path_queries, nentity, nrelation, args.negative_sample_size, train_answers),
                                    batch_size=args.batch_size,
                                    shuffle=True,
                                    num_workers=args.cpu_num,
                                    collate_fn=TrainDataset.collate_fn
                                ))
        if len(train_other_queries) > 0:
            train_other_queries = flatten_query(train_other_queries)
            train_other_iterator = \
                SingledirectionalOneShotIterator(DataLoader(
                TrainDataset(train_other_queries, nentity, nrelation, args.negative_sample_size, train_answers),
                batch_size=args.batch_size, shuffle=True, num_workers=args.cpu_num, collate_fn=TrainDataset.collate_fn ))
        else: train_other_iterator = None
    
    ''' Save validation related info '''
    logger.info("Validation info:")
    if args.do_valid:
        for query_structure in valid_queries:
            logger.info(query_name_dict[query_structure]+": "+str(len(valid_queries[query_structure])))
        
        ''' Convert queries into DataLoader '''
        valid_queries = flatten_query(valid_queries)
        valid_dataloader = DataLoader(
            TestDataset(
                valid_queries, 
                args.nentity, 
                args.nrelation, 
            ), 
            batch_size=args.test_batch_size,
            num_workers=args.cpu_num, 
            collate_fn=TestDataset.collate_fn
        )

    ''' Save test related info '''
    logger.info("Test info:")
    if args.do_test: 
        for query_structure in test_queries:
            logger.info(query_name_dict[query_structure]+": "+str(len(test_queries[query_structure])))
        
        ''' create test DataLoader '''
        test_queries = flatten_query(test_queries)
        test_dataloader = DataLoader(
            TestDataset(
                test_queries, 
                args.nentity, 
                args.nrelation, 
            ), 
            batch_size=args.test_batch_size,
            num_workers=args.cpu_num, 
            collate_fn=TestDataset.collate_fn
        )

    ''' Creating the model '''
    model = KGReasoning(
        nentity=nentity,
        nrelation=nrelation,
        hidden_dim=args.hidden_dim,
        gamma=args.gamma,
        geo=args.geo,
        use_cuda = args.cuda,
        box_mode=eval_tuple(args.box_mode),
        beta_mode = eval_tuple(args.beta_mode),
        test_batch_size=args.test_batch_size,
        query_name_dict = query_name_dict
    )

    ''' Recording parameters '''
    logger.info('Model Parameter Configuration:')
    num_params = 0
    for name, param in model.named_parameters():
        logger.info('Parameter %s: %s, require_grad = %s' % (name, str(param.size()), str(param.requires_grad)))
        if param.requires_grad:
            num_params += np.prod(param.size())
    logger.info('Parameter Number: %d' % num_params)

    if args.cuda: model = model.cuda()

    ''' Create training optimizer '''
    if args.do_train:
        current_learning_rate = args.learning_rate
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, model.parameters()), 
            lr=current_learning_rate )
        warm_up_steps = args.max_steps // 2

    ''' load checkpoint if specified '''
    if args.checkpoint_path is not None:
        logger.info('Loading checkpoint %s...' % args.checkpoint_path)
        checkpoint = torch.load(os.path.join(args.checkpoint_path, 'checkpoint'))
        init_step = checkpoint['step']
        model.load_state_dict(checkpoint['model_state_dict'])

        if args.do_train:
            current_learning_rate = checkpoint['current_learning_rate']
            warm_up_steps = checkpoint['warm_up_steps']
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    else:
        logger.info('Ramdomly Initializing %s Model...' % args.geo)
        init_step = 0

    ''' Print training info  '''
    step = init_step 
    if args.geo == 'beta':
        logger.info('beta mode = %s' % args.beta_mode)
    logger.info('tasks = %s' % args.tasks)
    logger.info('init_step = %d' % init_step)
    if args.do_train:
        logger.info('Start Training...')
        logger.info('learning_rate = %d' % current_learning_rate)
    logger.info('batch_size = %d' % args.batch_size)
    logger.info('hidden_dim = %d' % args.hidden_dim)
    logger.info('gamma = %f' % args.gamma)
    
    ''' Actual Training '''
    if args.do_train:
        training_logs = []

        for step in range(init_step, args.max_steps): # Training Loop
            if step == 2*args.max_steps//3:
                args.valid_steps *= 4
            log = model.train_step(model, optimizer, train_path_iterator, args, step)
            for metric in log: 
                writer.add_scalar('path_'+metric, log[metric], step)
            if train_other_iterator is not None:
                log = model.train_step(model, optimizer, train_other_iterator, args, step)
                for metric in log:
                    writer.add_scalar('other_'+metric, log[metric], step)
                log = model.train_step(model, optimizer, train_path_iterator, args, step)

            training_logs.append(log)

            if step >= warm_up_steps:
                current_learning_rate = current_learning_rate / 5
                logger.info('Change learning_rate to %f at step %d' % (current_learning_rate, step))
                optimizer = torch.optim.Adam(
                    filter(lambda p: p.requires_grad, model.parameters()), 
                    lr=current_learning_rate
                )
                warm_up_steps = warm_up_steps * 1.5
            
            if step % args.save_checkpoint_steps == 0:
                save_variable_list = {
                    'step': step, 
                    'current_learning_rate': current_learning_rate,
                    'warm_up_steps': warm_up_steps
                }
                save_model(model, optimizer, save_variable_list, args)

            if step % args.valid_steps == 0 and step > 0:
                if args.do_valid:
                    logger.info('Evaluating on Valid Dataset...')
                    valid_all_metrics = evaluate(model, valid_easy_answers, valid_hard_answers, args, valid_dataloader, query_name_dict, 'Valid', step, writer)

                if args.do_test:
                    logger.info('Evaluating on Test Dataset...')
                    test_all_metrics = evaluate(model, test_easy_answers, test_hard_answers, args, test_dataloader, query_name_dict, 'Test', step, writer)
                
            if step % args.log_steps == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    metrics[metric] = sum([log[metric] for log in training_logs])/len(training_logs)

                log_metrics('Training average', step, metrics)
                training_logs = []

        save_variable_list = {
            'step': step, 
            'current_learning_rate': current_learning_rate,
            'warm_up_steps': warm_up_steps
        }
        save_model(model, optimizer, save_variable_list, args)
        
    try:
        print (step)
    except:
        step = 0

    if args.do_test:
        logger.info('Evaluating on Test Dataset...')
        test_all_metrics = evaluate(model, test_easy_answers, test_hard_answers, args, test_dataloader, query_name_dict, 'Test', step, writer)

    logger.info("Training finished!!")

if __name__ == '__main__':
    main(args)

logging to logs\FB15k-betae\1p_2p_3p_2i_3i_ip_pi_2in_3in_inp_pin_pni_2u_up\beta\g_60.0_mode_(1600_2)\20211115074520


NameError: name 'load_data' is not defined